# Combine our filtered charges with our sentencing information

In [14]:
import pandas as pd
import numpy as np
import datetime
import csv

In [17]:
crim_sentence = pd.read_excel('../data/steps/Step2Output_sentences.xlsx')
crim_sentence.head()

,charge_id,disposition,disposition_date,judge,sentence,sentence_conditions,sentence_date,sentence_terms
0,3976200.0,No Information Signed,2008-05-28,"Erickson (Retired), Stephen",NaN,NaN,NaT,
1,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
2,5512512.0,Nolo Contendere Plea To Amended Charge,1999-03-26,NaN,Criminal Sentence,Suspended 15Y Judge: JUDGE PFEIFFER,1999-03-26,
3,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
4,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN


In [19]:
filtered_charges = pd.read_excel('../data/steps/Step1Output_charges.xlsx')
filtered_charges.head()

,Case ID,Case No,Charge ID,Charge Number,Offense Description,Offense Date,Charge Filing Date,Arresting PD
5,7422794,21-2001-01905,3909041,1,Larceny < 500 Person 65+\n,2001-08-03,2001-08-04,MIDDLETOWN POLICE DEPARTMENT
11,7423292,21-2001-02507,3909791,1,Larceny < 500 Person 65+\n,2001-09-24,2001-09-25,TIVERTON POLICE DEPARTMENT
35,7424561,21-2002-00802,3911694,2,FAILURE TO APPEAR FOR SUMMONS,2002-03-27,2002-03-27,Newport Police Department
36,7424561,21-2002-00802,3911693,1,Larceny < 500 Person 65+\n,2002-02-28,2002-03-27,Newport Police Department
54,7425369,21-2002-01767,3913116,1,Larceny < 500 Person 65+\n,2002-07-11,2002-07-24,Newport Police Department


## Sub-step 1: Ensure all filtered charges have a matching sentence

#### First, let's create a mapping from Charge ID to sentence

In [20]:
charge_to_sentence_map = {}

In [21]:
for row in crim_sentence.iterrows():
    charge_to_sentence_map[row[1]['charge_id']] = row[1]

#### Now, let's make sure every filtered charge has a matching sentence

In [42]:
total = 0
elder_unmatched = 0
matched = []
for row in filtered_charges.iterrows():
    if row[1]['Charge ID'] in charge_to_sentence_map:
        matched.append(row[1])
    else:
        elder_kws = ('elder','60','65')
        if any(kw in row[1]['Offense Description'] for kw in elder_kws):
            elder_unmatched += 0
    total += 1

In [44]:
print '{}/{} filtered charges could be mapped by Charge ID'.format(len(matched),total)
print '{}/{} filtered charges could NOT be mapped by Charge ID, but only {} of those were elder charges'.format(
    total-len(matched),total,elder_unmatched
)

3328/3336 filtered charges could be mapped by Charge ID
8/3336 filtered charges could NOT be mapped by Charge ID, but only 0 of those were elder charges


#### Okay, so they're not all matchable by Charge ID, but only 8 aren't and they aren't even elder abuse charges, so good to go matching by charge ID

## Sub-step 2: Create full charge + sentencing record for filtered cases

In [47]:
def snake_case(string):
    return string.lower().replace(' ', '_')

In [63]:
def create_merged_blob(charge, sentence):
    blob = {}
    for key in charge.keys():
        blob[snake_case(key)] = charge[key]
    for key in sentence.keys():
        blob[snake_case(key)] = sentence[key]
    return blob

In [64]:
blobs = []
for charge in matched:
    sentence = charge_to_sentence_map[charge['Charge ID']]
    full_blob = create_merged_blob(charge, sentence)
    blobs.append(full_blob)

In [65]:
len(blobs)

3328

In [66]:
to_write = pd.DataFrame(blobs)

## Sub-step 3: Write data to `steps` folder

In [67]:
writer = pd.ExcelWriter('../data/steps/Step3Output_charges_sentences.xlsx')
to_write.to_excel(writer,'Sheet1')
writer.save()